In [86]:
import h5py
import json, csv
import os
from pathlib import Path
import math
import pandas as pd
import numpy as np

import torch 
from transformers import AutoTokenizer
from elasticsearch_dsl import Document, Integer, Text, DenseVector
from elasticsearch_dsl.connections import connections

import time
import requests
from tqdm import tqdm
from dataclasses import dataclass

from dataclasses_json import dataclass_json
from dacite import from_dict
from nltk import tokenize
from omegaconf import OmegaConf 
from pprint import pprint

from transformers import DistilBertTokenizer
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence

import src.data
from src.data import load_metadata, find_paths, relative_file_path

import sys
sys.path.append('../contrastive_mm2/') 
from gru_test2 import mmModule, Cfg
from prepare_index_sentencelevel2 import read_metadata_subset

# Load static configuration variables. 
config_path = "./config.yaml"
conf = OmegaConf.load(config_path)
print("[cudacheck] Is cuda available? ", torch.cuda.is_available())

[cudacheck] Is cuda available?  False
[cudacheck] Is cuda available?  False


In [87]:
model_path = '/Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/contrastive_mm2/logs/windows_gru2-clip_loss_followup'
# model_path = "E:\msc_thesis\code\contrastive_mm2\logs\windows_gru2-clip_loss_followup"

transcripts_path = '/Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/data/sp/podcasts-no-audio-13GB/podcasts-transcripts'
# transcripts_path = 'E:/msc_thesis/code/data/sp/podcasts-no-audio-13GB/podcasts-transcripts'

# Whether we create for the train or the tes split. 
traintest = 'test'


# Creating output directory.
save_name = os.path.split(model_path)[-1]
save_path = os.path.join(conf.yamnet_topic_embed_path, save_name + "_" + traintest)
Path(save_path).mkdir(parents=True, exist_ok=True)

# Reading the metadata.
metadata_testset, topics_df, topics_df_targets = read_metadata_subset(conf, traintest='test')

# Remove duplicate rows
metadata_testset = metadata_testset.drop_duplicates(subset=['episode_filename_prefix']).sort_values('episode_filename_prefix')
print("[main] Topic test set loaded: ", len(metadata_testset))

# Loading the model.
print("[Load model] from ", model_path)
model_weights = os.path.join(model_path, "output/full_model_weights.pth")
print("[Load model] weights loaded")
model_config_path = os.path.join(model_path, 'config.json')

# Opening JSON file
f = open(model_config_path)
model_config = json.load(f)
print("using config: , ", model_config)
print("\n\n")
CFG = from_dict(data_class=Cfg, data=model_config)
CFG.device = "cuda" if torch.cuda.is_available() else "cpu"
# mutual_embed_dim = CFG.final_projection_dim
print("[Load model] config loaded: ", CFG)


tokenizer = AutoTokenizer.from_pretrained(CFG.text_model_name)


# Create dataloader of test set
data_loader = MMloader(CFG)
# train_loader = data_loader.train_loader
# val_loader = data_loader.val_loader
test_loader = data_loader.test_loader # Not used!

# TODO: move this into __init__ of dataloader
data_loader.test_dataset.tokenizer = tokenizer
data_loader.test_dataset.text_max_length = CFG.text_max_length

# Load model and tokenizer
full_model = mmModule(CFG)
print("[del] device: ", CFG.device)
full_model.load_state_dict(torch.load(model_weights,  map_location=CFG.device))              
# full_model.load_state_dict(torch.load(model_weights))   
full_model = full_model.to(CFG.device)     
full_model.eval()


[main] metadata loaded  105360
[main] topics_test loaded  50
[main] topics_df_targets loaded for test set 9426
[main] metadata_subset loaded  9426
[main] Topic test set loaded:  3421
[Load model] from  /Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/contrastive_mm2/logs/windows_gru2-clip_loss_followup
[Load model] weights loaded
using config: ,  {'audio_dropout': 0.1, 'audio_encoder_input': 1024, 'audio_hidden_dim': 768, 'audio_layer_dim': 2, 'device': 'cuda', 'lr': 5e-05, 'mutual_embedding_dim': 768, 'text_dropout': 0.1, 'text_max_length': 32, 'text_model_name': 'distilbert-base-uncased', 'text_tokenizer': 'distilbert-base-uncased', 'weight_decay': 0.01, 'audio_activation': 'gelu', 'audio_proj_head': 'gru', 'batch_size': 128, 'cuda_number': '0', 'dev_dataset': 'sts', 'eval_every': 100, 'final_projection_dim': 768, 'fp16': False, 'load_checkpoint': False, 'load_checkpoint_path': './logs/load_test2/output/checkpoint.pth', 'load_model': False, 'load_model_path': './logs/l

[Transformer_multimodal] Configurations used: 
 DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[del] device:  cpu


mmModule(
  (text_model): Sequential(
    (0): TextEncoder(
      (auto_model): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0): TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in_features=768, out_features=768, bias=True)
              )
              (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_

In [88]:
print("[Create embeds] start for TEST")
iterator = iter(test_loader)
max_steps = len(test_loader) 
processed_text = []
processed_audio = []

topic_norm_reps_text = []
topic_norm_reps_audio = []
matrix_targets = []
vali_accs = []

# max_steps = 100 ## DEL

sent_representations = np.zeros((max_steps * 128, 768)) # TODO: mutual embed dim
audio_representations = np.zeros((max_steps * 128, 768))# TODO: mutual embed dim



[Create embeds] start for TEST


In [90]:
for step in range(max_steps):
    print("{}/{}".format(step, max_steps))
    if step > 5:
        print('break for now')
        break
    batch = next(iterator)

    (tok_sentences, audio_features, seq_len, targets) = batch

    with torch.no_grad():


        reps_sentences = full_model.text_model(tok_sentences)['sentence_embedding']
        reps_audio = full_model.audio_model((audio_features, seq_len))

        # Normalized representations
        norm_reps_sentences = reps_sentences / reps_sentences.norm(dim=1, keepdim=True)
        norm_reps_audio = reps_audio / reps_audio.norm(dim=1, keepdim=True)

        # This will be returned
        idxs = np.arange(step * 128, step*128+128)
        sent_representations[idxs, :] = norm_reps_sentences.cpu().numpy().copy()
        audio_representations[idxs, :] = norm_reps_audio.cpu().numpy().copy()
        matrix_targets.extend(targets)

        # This has to be deleted
        # topic_norm_reps_text.append(reps_sentences / reps_sentences.norm(dim=1, keepdim=True))
        # topic_norm_reps_audio.append(reps_audio / reps_audio.norm(dim=1, keepdim=True))

        # Calculate accuracy of test set
        audio_logits =  (reps_audio @ reps_sentences.t()) * CFG.scale
        text_logits = audio_logits.t()

        probs = audio_logits.softmax(dim=-1).cpu().numpy()
        ground_truth = torch.arange(128)
        acc = torch.eq(torch.tensor(probs.argmax(axis=0)), ground_truth).sum() / ground_truth.shape[0]
        print("accuracy", acc.item())
        vali_accs.append(acc.item())

print("This was acc: ", np.mean(vali_accs))

# TODO: THis results in CUDA out oOOM
# topic_norm_reps_audio = torch.cat(topic_norm_reps_audio, dim=0)
# topic_norm_reps_text = torch.cat(topic_norm_reps_text, dim=0)
# print(topic_norm_reps_audio.shape, topic_norm_reps_text.shape)
# print(sent_representations.shape, audio_representations.shape)

topic_norm_reps_text = sent_representations
topic_norm_reps_audio = audio_representations
print("[Create embeds] Done")

0/4105
accuracy 0.5234375
1/4105
accuracy 0.515625
2/4105
accuracy 0.4609375
3/4105
accuracy 0.5234375
4/4105
accuracy 0.46875
5/4105
accuracy 0.5703125
6/4105
break for now
This was acc:  0.5104166666666666
[Create embeds] Done


In [92]:
ids_we_can_use = [m.split('_')[0] for m in matrix_targets]
print("WE CAN USE {} ids".format(len(set(ids_we_can_use))))
val_df = topics_df_targets[topics_df_targets['episode_uri'].isin(ids_we_can_use)]

val_df.head()

WE CAN USE 512 ids


,num,unk,episode_uri_time,score,bin_score,episode_uri,time
0,9,0,0BrvXC9DNmicnpCtHmFltv_2220,0,0,0BrvXC9DNmicnpCtHmFltv,2220
1,9,0,0BrvXC9DNmicnpCtHmFltv_2280,0,0,0BrvXC9DNmicnpCtHmFltv,2280
2,9,0,0BrvXC9DNmicnpCtHmFltv_3120,0,0,0BrvXC9DNmicnpCtHmFltv,3120
3,9,0,0BrvXC9DNmicnpCtHmFltv_3180,0,0,0BrvXC9DNmicnpCtHmFltv,3180
15,9,0,0YYgNb2r5zngCM5VLMpqEb_60,0,0,0YYgNb2r5zngCM5VLMpqEb,60


In [96]:
positive_episodes = val_df.loc[val_df['bin_score'] == 1].copy()
positive_eplist = positive_episodes['episode_uri'].tolist()

for i, row in val_df.iterrows():
    ifor_val = 0
    if row['episode_uri'] in positive_eplist:
        ifor_val = 1
    val_df.loc[i,'ep_score'] = ifor_val

val_df.ep_score = val_df.ep_score.astype(int)
print(len(val_df))
val_df.head(12)

/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


1281


/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,num,unk,episode_uri_time,score,bin_score,episode_uri,time,ep_score
0,9,0,0BrvXC9DNmicnpCtHmFltv_2220,0,0,0BrvXC9DNmicnpCtHmFltv,2220,0
1,9,0,0BrvXC9DNmicnpCtHmFltv_2280,0,0,0BrvXC9DNmicnpCtHmFltv,2280,0
2,9,0,0BrvXC9DNmicnpCtHmFltv_3120,0,0,0BrvXC9DNmicnpCtHmFltv,3120,0
3,9,0,0BrvXC9DNmicnpCtHmFltv_3180,0,0,0BrvXC9DNmicnpCtHmFltv,3180,0
15,9,0,0YYgNb2r5zngCM5VLMpqEb_60,0,0,0YYgNb2r5zngCM5VLMpqEb,60,0
33,9,0,1nu9RsgfYsbNb7CDmHlvKx_1260,0,0,1nu9RsgfYsbNb7CDmHlvKx,1260,1
34,9,0,1nu9RsgfYsbNb7CDmHlvKx_1320,2,1,1nu9RsgfYsbNb7CDmHlvKx,1320,1
35,9,0,1nu9RsgfYsbNb7CDmHlvKx_1380,2,1,1nu9RsgfYsbNb7CDmHlvKx,1380,1
36,9,0,1nu9RsgfYsbNb7CDmHlvKx_1500,0,0,1nu9RsgfYsbNb7CDmHlvKx,1500,1
37,9,0,1nu9RsgfYsbNb7CDmHlvKx_1680,0,0,1nu9RsgfYsbNb7CDmHlvKx,1680,1


In [97]:
topics_df.head(3)

,num,query,type,description
0,9,trump call ukrainian president,topical,The White House released a rough transcript of...
1,10,boeing 737 crash causes,topical,The Boeing 737 MAX passenger airliner was grou...
2,11,how to cook turkey,topical,I’m looking for podcasts that talk about how t...


In [139]:

print("Tokenizing queries")
query_field =  'query' # TODO: ook description?
queries = topics_df[query_field].tolist()
tokenized_queries = tokenizer(
    queries, padding=True, truncation=True, max_length=32, return_tensors='pt', return_token_type_ids=True,
)

print("Creating padding of yamnets of queries")
query_yamnets = []
query_lengths = []
for idx, row in topics_df.iterrows():
    query_num = row.num
#     print("query_num: ", query_num)
    query_embed_path  = os.path.join(conf.yamnet_query_embed_path, str(query_num) + ".h5")
    inbetween_yamnet_embeds = pd.read_hdf(query_embed_path)

    query_lengths.append(len(inbetween_yamnet_embeds))
    tmp = torch.Tensor(inbetween_yamnet_embeds.values)
    query_yamnets.append(tmp)
    #lengths.append(len(example[1]))

padded_query_yamnets = pad_sequence(query_yamnets, batch_first=True).to(CFG.device)
print(padded_query_yamnets.shape)


print("Creating query embeddings now:")
query_text_repr = []
query_audio_repr = []
query_field = 'query'

query_norm_reps_audio

with torch.no_grad():
    # print("[del] get embeds: ")
    reps_sentences = full_model.text_model(tokenized_queries)['sentence_embedding']
    reps_audio = full_model.audio_model((padded_query_yamnets, query_lengths))

    # audio_logits =  (reps_audio @ reps_sentences.t()) * CFG.scale
    # text_logits = audio_logits.t()

    # audio_logits = audio_logits / audio_logits.norm(dim=1, keepdim=True)
    # text_logits = text_logits / text_logits.norm(dim=1, keepdim=True)
    #probs = audio_logits.softmax(dim=-1).cpu().numpy()
    # query_text_logits = text_logits
    # query_audio_logits = audio_logits

    query_norm_reps_audio = reps_audio / reps_audio.norm(dim=1, keepdim=True)
    query_norm_reps_text = reps_sentences / reps_sentences.norm(dim=1, keepdim=True)

    query_text_repr.append(query_norm_reps_text)
    query_audio_repr.append(query_norm_reps_audio)
    


query_audio_repr = torch.cat(query_audio_repr, dim=0).cpu()
query_text_repr = torch.cat(query_text_repr, dim=0).cpu()


print("Shapes: ", query_audio_repr.shape, query_audio_repr.shape)
print(": ", topic_norm_reps_text.shape)
print("----------")
print("TOPK")


Tokenizing queries
Creating padding of yamnets of queries
torch.Size([50, 7, 1024])
Creating query embeddings now:
Shapes:  torch.Size([50, 768]) torch.Size([50, 768])
:  (525440, 768)
----------
TOPK


In [145]:
k = 50
pred_episodes = {}
results = {}

combis = [[query_text_repr, topic_norm_reps_audio], [query_audio_repr, topic_norm_reps_audio], [query_audio_repr, topic_norm_reps_audio]]
names = ["text2text", "audio2audio", "text2audio"]
for idx, combi in enumerate(combis):
    name = names[idx]
    results[name] = defaultdict(list)
    
    similarity = 100 * query_audio_repr @ topic_norm_reps_text.T
    probs = similarity.softmax(dim=-1).cpu()

    top_probs, top_labels = probs.topk(k, dim=-1)
    
    for row_idx in range(len(topics_df)):
        query = topics_df[query_field][row_idx]
        pred_ind = top_labels[row_idx].tolist()

        pred_epis = [matrix_targets[val].split('_')[0] for val in pred_ind]
        query_num = topics_df['num'][row_idx]
        print("checking for {} and num {} ".format(row_idx, query_num))

        tmp = val_df[(val_df.num == query_num) & (val_df.ep_score==1)]
        tmp = tmp['episode_uri'].tolist()
        num_episodes_relevant = len(set(tmp))
        print("[del] Relevant episodes for Query: ", num_episodes_relevant)


        ep_scores = []
        for episode_uri in pred_epis:
            ep_score_row = val_df.loc[val_df['episode_uri'] == episode_uri]
            if len(ep_score_row) > 0 and ep_score_row.num.values[0] == query_num:
                ep_scores.append(1)
            else:
                ep_scores.append(0)


#         pred_episodes[query] = {}
#         # pred_episodes[query]['episodes'] = [matrix_targets[val].split('_')[0] for val in pred_ind]
#         pred_episodes[query]['ep_score'] = ep_scores

#         pred_episodes[query]['prec@3'] = precision_at_k(ep_scores, 3)
#         pred_episodes[query]['prec@10'] = precision_at_k(ep_scores, 10)
#         pred_episodes[query]['prec@30'] = precision_at_k(ep_scores, 30)




        targets = [[1] * num_episodes_relevant + [0] * (len(ep_scores) - num_episodes_relevant)]
        #print("input ndcg: ", targets)
        #print("and also: ", ep_scores)
        ndcg_ep_score = ndcg_score(targets, [ep_scores], k=30)
        #pred_episodes[query]['ndcg'] = ndcg_ep_score
        #print("done query {}, p@10 {}, ndcg: {}".format(query_num, pred_episodes[query]['prec@10'], ndcg_ep_score))

        results[name]['prec@3'].append(precision_at_k(ep_scores, 3))
        results[name]['prec@10'].append(precision_at_k(ep_scores, 10))
        results[name]['prec@30'].append(precision_at_k(ep_scores, 30))
        results[name]['ndcg'].append(ndcg_ep_score)



print(results)


checking for 0 and num 9 
[del] Relevant episodes for Query:  4
checking for 1 and num 10 
[del] Relevant episodes for Query:  5
checking for 2 and num 11 
[del] Relevant episodes for Query:  7
checking for 3 and num 12 
[del] Relevant episodes for Query:  0
checking for 4 and num 13 
[del] Relevant episodes for Query:  2
checking for 5 and num 14 
[del] Relevant episodes for Query:  3
checking for 6 and num 15 
[del] Relevant episodes for Query:  17
checking for 7 and num 16 
[del] Relevant episodes for Query:  8
checking for 8 and num 17 
[del] Relevant episodes for Query:  3
checking for 9 and num 18 
[del] Relevant episodes for Query:  6
checking for 10 and num 19 
[del] Relevant episodes for Query:  0
checking for 11 and num 20 
[del] Relevant episodes for Query:  3
checking for 12 and num 21 
[del] Relevant episodes for Query:  2
checking for 13 and num 22 
[del] Relevant episodes for Query:  1
checking for 14 and num 23 
[del] Relevant episodes for Query:  2
checking for 15 and 

checking for 36 and num 45 
[del] Relevant episodes for Query:  12
checking for 37 and num 46 
[del] Relevant episodes for Query:  0
checking for 38 and num 47 
[del] Relevant episodes for Query:  0
checking for 39 and num 48 
[del] Relevant episodes for Query:  1
checking for 40 and num 49 
[del] Relevant episodes for Query:  0
checking for 41 and num 50 
[del] Relevant episodes for Query:  0
checking for 42 and num 51 
[del] Relevant episodes for Query:  1
checking for 43 and num 52 
[del] Relevant episodes for Query:  1
checking for 44 and num 53 
[del] Relevant episodes for Query:  7
checking for 45 and num 54 
[del] Relevant episodes for Query:  1
checking for 46 and num 55 
[del] Relevant episodes for Query:  5
checking for 47 and num 56 
[del] Relevant episodes for Query:  1
checking for 48 and num 57 
[del] Relevant episodes for Query:  1
checking for 49 and num 58 
[del] Relevant episodes for Query:  2
{'text2text': defaultdict(<class 'list'>, {'prec@3': [0.0, 0.0, 0.0, 0.0, 0

In [146]:
mean_results = {}
for name, metric in results.items():
    mean_results[name] = {}
    for metric_name, vals in metric.items():
        mean_results[name][metric_name] = np.mean(vals)
        
print(mean_results)
        

{'text2text': {'prec@3': 0.02, 'prec@10': 0.018000000000000002, 'prec@30': 0.021333333333333333, 'ndcg': 0.22469550260137555}, 'audio2audio': {'prec@3': 0.02, 'prec@10': 0.018000000000000002, 'prec@30': 0.021333333333333333, 'ndcg': 0.22469550260137555}, 'text2audio': {'prec@3': 0.02, 'prec@10': 0.018000000000000002, 'prec@30': 0.021333333333333333, 'ndcg': 0.22469550260137555}}


In [217]:

# # we predict some scores (relevance) for the answers
# scores = np.array([[1, 0, 0, 0, 0]])
# ndcg_ep_score = ndcg_score(true_relevance, scores, k=30)
# print(ndcg_ep_score)

from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
                  
# ap = average_precision_score(true_relevance, scores)
# print("ap=",ap)

# roc = roc_auc_score(true_relevance, scores)
# print("roc=",roc)


targets =  [[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
scores =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    
# if  all(np.array(targets[0]) == 0) or all(np.array(scores) ==0):
#     ap = 0.0
# else:
#     ap = average_precision_score(targets[0], scores)
# print(ap)
# if np.isnan(ap):
#     print("ja")
    
    
ndcg_ep_score = ndcg_score(targets, [scores], k=30)
print(ndcg_ep_score)
    
    
roc = roc_auc_score(targets[0], scores)
print("roc=",roc)
m = 4


test = average_precision_score(targets[0][:m], scores[:m])
print(test)



0.2600912991941578
roc= 0.4891304347826087
0.75


In [220]:
t = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
s = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


roc = roc_auc_score(t, s)

print(roc)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [112]:
# for row_idx in range(len(topics_df)):
#     query = topics_df[query_field][row_idx]
#     pred_ind = top_labels[row_idx].tolist()

#     pred_epis = [matrix_targets[val].split('_')[0] for val in pred_ind]
#     query_num = topics_df['num'][row_idx]
#     print("checking for {} and num {} ".format(row_idx, query_num))
    
#     tmp = val_df[(val_df.num == query_num) & (val_df.ep_score==1)]
#     tmp = tmp['episode_uri'].tolist()
#     num_episodes_relevant = len(set(tmp))
#     print("[del] Relevant episodes for Query: ", num_episodes_relevant)


#     ep_scores = []
#     for episode_uri in pred_epis:
#         ep_score_row = val_df.loc[val_df['episode_uri'] == episode_uri]
#         if len(ep_score_row) > 0 and ep_score_row.num.values[0] == query_num:
#             ep_scores.append(1)
#         else:
#             ep_scores.append(0)
            

#     pred_episodes[query] = {}
#     # pred_episodes[query]['episodes'] = [matrix_targets[val].split('_')[0] for val in pred_ind]
#     pred_episodes[query]['ep_score'] = ep_scores

#     pred_episodes[query]['prec@3'] = precision_at_k(ep_scores, 3)
#     pred_episodes[query]['prec@10'] = precision_at_k(ep_scores, 10)
#     pred_episodes[query]['prec@30'] = precision_at_k(ep_scores, 30)
    
    


#     targets = [[1] * num_episodes_relevant + [0] * (len(ep_scores) - num_episodes_relevant)]
#     #print("input ndcg: ", targets)
#     #print("and also: ", ep_scores)
#     ndcg_ep_score = ndcg_score(targets, [ep_scores], k=30)
#     pred_episodes[query]['ndcg'] = ndcg_ep_score
#     #print("done query {}, p@10 {}, ndcg: {}".format(query_num, pred_episodes[query]['prec@10'], ndcg_ep_score))
    
#     results['prec@3'].append(precision_at_k(ep_scores, 3))
#     results['prec@10'].append(precision_at_k(ep_scores, 10))
#     results['prec@30'].append(precision_at_k(ep_scores, 30))
#     results['ndcg'].append(ndcg_ep_score)





checking for 0 and num 9 
[del] Relevant episodes for Query:  4
checking for 1 and num 10 
[del] Relevant episodes for Query:  5
checking for 2 and num 11 
[del] Relevant episodes for Query:  7
checking for 3 and num 12 
[del] Relevant episodes for Query:  0
checking for 4 and num 13 
[del] Relevant episodes for Query:  2
checking for 5 and num 14 
[del] Relevant episodes for Query:  3
checking for 6 and num 15 
[del] Relevant episodes for Query:  17
checking for 7 and num 16 
[del] Relevant episodes for Query:  8
checking for 8 and num 17 
[del] Relevant episodes for Query:  3
checking for 9 and num 18 
[del] Relevant episodes for Query:  6
checking for 10 and num 19 
[del] Relevant episodes for Query:  0
checking for 11 and num 20 
[del] Relevant episodes for Query:  3
checking for 12 and num 21 
[del] Relevant episodes for Query:  2
checking for 13 and num 22 
[del] Relevant episodes for Query:  1
checking for 14 and num 23 
[del] Relevant episodes for Query:  2
checking for 15 and 

In [137]:
from collections import defaultdict
ep_scores = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
num_episodes_relevant = 6
targets = [[1] * num_episodes_relevant + [0] * (len(ep_scores) - num_episodes_relevant)]
# targets = [targets[0][:k]]
print("input ndcg: ", targets)
print("and also: ", ep_scores)
ndcg_ep_score = ndcg_score(targets, [ep_scores], k=30)
print(ndcg_ep_score)


mets = ['prec@3','prec@10','prec@30','ndcg']
full_mets = defaultdict(list)
mean_mets = defaultdict(list)
for query, qval in pred_episodes.items():
    query_metrics = pred_episodes[query]
    for m in (mets):
        full_mets[m].append(query_metrics[m])
    

for k, v in full_mets.items():
    mean_mets[k] = np.mean(v)
    
print(mean_mets)

input ndcg:  [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
and also:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.2848491568810346
defaultdict(<class 'list'>, {'prec@3': 0.02, 'prec@10': 0.018000000000000002, 'prec@30': 0.021333333333333333, 'ndcg': 0.22469550260137555})


In [36]:

# Creating output directory.
save_name = os.path.split(model_path)[-1]
save_path = os.path.join(conf.yamnet_topic_embed_path, save_name + "_" + traintest)
Path(save_path).mkdir(parents=True, exist_ok=True)

# Reading the metadata.
metadata_testset, topics_df, topics_df_targets = read_metadata_subset(conf, traintest='test')

# Remove duplicate rows
metadata_testset = metadata_testset.drop_duplicates(subset=['episode_filename_prefix']).sort_values('episode_filename_prefix')
print("[main] Topic test set loaded: ", len(metadata_testset))


[main] metadata loaded  105360
[main] topics_test loaded  50
[main] topics_df_targets loaded for test set 9426
[main] metadata_subset loaded  9426
[main] Topic test set loaded:  3421


In [37]:

# Loading the model.
print("[Load model] from ", model_path)
model_weights = os.path.join(model_path, "output/full_model_weights.pth")
print("[Load model] weights loaded")
model_config_path = os.path.join(model_path, 'config.json')

# Opening JSON file
f = open(model_config_path)
model_config = json.load(f)
print("using config: , ", model_config)
print("\n\n")
CFG = from_dict(data_class=Cfg, data=model_config)
CFG.device = "cuda" if torch.cuda.is_available() else "cpu"
# mutual_embed_dim = CFG.final_projection_dim
print("[Load model] config loaded: ", CFG)

[Load model] from  /Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/contrastive_mm2/logs/windows_gru2-clip_loss_followup
[Load model] weights loaded
using config: ,  {'audio_dropout': 0.1, 'audio_encoder_input': 1024, 'audio_hidden_dim': 768, 'audio_layer_dim': 2, 'device': 'cuda', 'lr': 5e-05, 'mutual_embedding_dim': 768, 'text_dropout': 0.1, 'text_max_length': 32, 'text_model_name': 'distilbert-base-uncased', 'text_tokenizer': 'distilbert-base-uncased', 'weight_decay': 0.01, 'audio_activation': 'gelu', 'audio_proj_head': 'gru', 'batch_size': 128, 'cuda_number': '0', 'dev_dataset': 'sts', 'eval_every': 100, 'final_projection_dim': 768, 'fp16': False, 'load_checkpoint': False, 'load_checkpoint_path': './logs/load_test2/output/checkpoint.pth', 'load_model': False, 'load_model_path': './logs/load_test2/output/full_model_weights.pth', 'log_dir': './logs', 'loss_type': 'clip_loss', 'no_cuda': False, 'normalize': True, 'num_epochs': 10, 'pad_pack': True, 'save_checkpoint': Tr

In [38]:
# Load model and tokenizer
full_model = mmModule(CFG)
full_model.load_state_dict(torch.load(model_weights,  map_location=CFG.device))                      
full_model.eval()

tokenizer = AutoTokenizer.from_pretrained(CFG.text_model_name)
# tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
print("[Load model] model loaded: ", model_weights)
print(full_model.state_dict()['audio_model.0.seq_model.weight_ih_l1'][0][0])

[Transformer_multimodal] Configurations used: 
 DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[Load model] model loaded:  /Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/contrastive_mm2/logs/windows_gru2-clip_loss_followup/output/full_model_weights.pth
tensor(0.0674)


In [39]:

%load_ext autoreload
%autoreload 2

from gru_test2 import MMloader

# Create dataloader of test set
data_loader = MMloader(CFG)
train_loader = data_loader.train_loader
val_loader = data_loader.val_loader
test_loader = data_loader.test_loader # Not used!

# TODO: move this into __init__ of dataloader
data_loader.test_dataset.tokenizer = tokenizer
data_loader.test_dataset.text_max_length = CFG.text_max_length




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[MMloader] train dataset  sp
[spDataset] init from directory  ../data/sp/yamnet/processed/ train
[spDataset] found 1 h5py files
[del] h5py_file:  ../data/sp/yamnet/processed/train/0_0_embeds.h5
[MMloader] train dataset loaded, length:  19344
[spDataset] init from directory  ../data/sp/yamnet/processed/ val
[spDataset] found 1 h5py files
[del] h5py_file:  ../data/sp/yamnet/processed/val/0_0_embeds.h5
[spDataset] init from directory  ../data/sp/yamnet/processed/ test
[spDataset] found 97 h5py files
[del] h5py_file:  ../data/sp/yamnet/processed/test/0_N_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/1_R_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/2_1_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/1_G_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/2_M_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/0_2_embeds.h5
[del] h5py_file:  ..

## Standard validation

In [43]:
iterator = iter(test_loader)
max_steps = len(test_loader) 
print(max_steps)


for step in range(max_steps):
    print("step {}/{}".format(step, max_steps))
    batch = next(iterator)
    
    (tok_sentences, audio_features, seq_len, targets) = batch

    with torch.no_grad():
        reps_sentences = full_model.text_model(tok_sentences)['sentence_embedding']
        reps_audio = full_model.audio_model((audio_features, seq_len))
        
        audio_logits =  (reps_audio @ reps_sentences.t()) * CFG.scale
        text_logits = audio_logits.t()
        
        audio_logits = audio_logits / audio_logits.norm(dim=1, keepdim=True)
        text_logits = text_logits / text_logits.norm(dim=1, keepdim=True)
        
        probs = audio_logits.softmax(dim=-1).cpu().numpy()


4105
step 0/4105
step 1/4105
step 2/4105
step 3/4105
step 4/4105
step 5/4105
step 6/4105
step 7/4105
step 8/4105
step 9/4105
step 10/4105
step 11/4105
step 12/4105
step 13/4105
step 14/4105
step 15/4105
step 16/4105
step 17/4105
step 18/4105
step 19/4105
step 20/4105
step 21/4105
step 22/4105
step 23/4105
step 24/4105
step 25/4105
step 26/4105
step 27/4105
step 28/4105
step 29/4105
step 30/4105
step 31/4105
step 32/4105
step 33/4105
step 34/4105
step 35/4105
step 36/4105
step 37/4105
step 38/4105
step 39/4105
step 40/4105
step 41/4105
step 42/4105
step 43/4105
step 44/4105
step 45/4105
step 46/4105
step 47/4105
step 48/4105
step 49/4105
step 50/4105
step 51/4105
step 52/4105
step 53/4105
step 54/4105
step 55/4105
step 56/4105
step 57/4105
step 58/4105
step 59/4105
step 60/4105
step 61/4105
step 62/4105
step 63/4105
step 64/4105
step 65/4105
step 66/4105
step 67/4105
step 68/4105
step 69/4105
step 70/4105
step 71/4105
step 72/4105
step 73/4105
step 74/4105
step 75/4105
step 76/4105
step

step 593/4105
step 594/4105
step 595/4105
step 596/4105
step 597/4105
step 598/4105
step 599/4105
step 600/4105
step 601/4105
step 602/4105
step 603/4105
step 604/4105
step 605/4105
step 606/4105
step 607/4105
step 608/4105
step 609/4105
step 610/4105
step 611/4105
step 612/4105
step 613/4105
step 614/4105
step 615/4105
step 616/4105
step 617/4105
step 618/4105
step 619/4105
step 620/4105
step 621/4105
step 622/4105
step 623/4105
step 624/4105
step 625/4105
step 626/4105
step 627/4105
step 628/4105
step 629/4105
step 630/4105
step 631/4105
step 632/4105
step 633/4105
step 634/4105
step 635/4105
step 636/4105
step 637/4105
step 638/4105
step 639/4105
step 640/4105
step 641/4105
step 642/4105
step 643/4105
step 644/4105
step 645/4105
step 646/4105
step 647/4105
step 648/4105
step 649/4105
step 650/4105
step 651/4105
step 652/4105
step 653/4105
step 654/4105
step 655/4105
step 656/4105
step 657/4105
step 658/4105
step 659/4105
step 660/4105
step 661/4105
step 662/4105
step 663/4105
step 6

KeyboardInterrupt: 

In [44]:
probs = audio_logits.softmax(dim=-1).cpu().numpy()
# print(probs.argmax(axis=0))
ground_truth = torch.arange(128)
# print(ground_truth)


print("accuracy: ")
acc = torch.eq(torch.tensor(probs.argmax(axis=0)), ground_truth).sum() / ground_truth.shape[0]
print(acc)

accuracy: 
tensor(0.5469)


## Create embeddings of shows in test set

In [76]:
iterator = iter(test_loader)
processed_text = []
processed_audio = []

topic_norm_reps_text = []
topic_norm_reps_audio = []

matrix_targets = []

In [77]:


for step in range(max_steps):
    print("{}/{}".format(step, max_steps))
    if step < 778:
        print('continue')
        continue
    batch = next(iterator)
    
    (tok_sentences, audio_features, seq_len, targets) = batch

    with torch.no_grad():
        # print("[del] get embeds: ")
        reps_sentences = full_model.text_model(tok_sentences)['sentence_embedding']
        # embeds = self.model.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        #reps_sentences = reps_sentences.detach().cpu().numpy()
        
        
        reps_audio = full_model.audio_model((audio_features, seq_len))
        
        # Normalized representations
        topic_norm_reps_text.append(reps_sentences / reps_sentences.norm(dim=1, keepdim=True))
        topic_norm_reps_audio.append(reps_audio / reps_audio.norm(dim=1, keepdim=True))
        
        #print(reps_audio.shape)
        audio_logits =  (reps_audio @ reps_sentences.t()) * CFG.scale
        text_logits = audio_logits.t()
        
        matrix_targets.extend(targets)
        
        
#         audio_logits = audio_logits / audio_logits.norm(dim=1, keepdim=True)
#         text_logits = text_logits / text_logits.norm(dim=1, keepdim=True)
        
#         #probs = audio_logits.softmax(dim=-1).cpu().numpy()
        
#         processed_text.append(audio_logits)
        
#         processed_audio.append(audio_logits)
        
#     print(probs)
print()

0/4105
1/4105
2/4105
3/4105
4/4105
5/4105
6/4105
7/4105
8/4105
9/4105
10/4105
11/4105
12/4105
13/4105
14/4105
15/4105
16/4105
17/4105
18/4105
19/4105
20/4105
21/4105
22/4105
23/4105
24/4105
25/4105
26/4105
27/4105
28/4105
29/4105
30/4105
31/4105
32/4105
33/4105
34/4105
35/4105
36/4105
37/4105
38/4105
39/4105
40/4105
41/4105
42/4105
43/4105
44/4105
45/4105
46/4105
47/4105
48/4105
49/4105
50/4105
51/4105
continue
52/4105
continue
53/4105
continue
54/4105
continue
55/4105
continue
56/4105
continue
57/4105
continue
58/4105
continue
59/4105
continue
60/4105
continue
61/4105
continue
62/4105
continue
63/4105
continue
64/4105
continue
65/4105
continue
66/4105
continue
67/4105
continue
68/4105
continue
69/4105
continue
70/4105
continue
71/4105
continue
72/4105
continue
73/4105
continue
74/4105
continue
75/4105
continue
76/4105
continue
77/4105
continue
78/4105
continue
79/4105
continue
80/4105
continue
81/4105
continue
82/4105
continue
83/4105
continue
84/4105
continue
85/4105
continue
86/4105

524/4105
525/4105
526/4105
527/4105
528/4105
529/4105
530/4105
531/4105
532/4105
533/4105
534/4105
535/4105
536/4105
537/4105
538/4105
539/4105
540/4105
541/4105
542/4105
543/4105
544/4105
545/4105
546/4105
547/4105
548/4105
549/4105
550/4105
551/4105
552/4105
553/4105
554/4105
555/4105
556/4105
557/4105
558/4105
559/4105
560/4105
561/4105
562/4105
563/4105
564/4105
565/4105
566/4105
567/4105
568/4105
569/4105
570/4105
571/4105
572/4105
573/4105
574/4105
575/4105
576/4105
577/4105
578/4105
579/4105
580/4105
581/4105
582/4105
583/4105
584/4105
585/4105
586/4105
587/4105
588/4105
589/4105
590/4105
591/4105
592/4105
593/4105
594/4105
595/4105
596/4105
597/4105
598/4105
599/4105
600/4105
601/4105
602/4105
603/4105
604/4105
605/4105
606/4105
607/4105
608/4105
609/4105
610/4105
611/4105
612/4105
613/4105
614/4105
615/4105
616/4105
617/4105
618/4105
619/4105
620/4105
621/4105
622/4105
623/4105
624/4105
625/4105
626/4105
627/4105
628/4105
629/4105
630/4105
631/4105
632/4105
633/4105
634/4105
6

KeyboardInterrupt: 

In [79]:
print(topic_norm_reps_audio[0].shape)

topic_norm_reps_audio = torch.cat(topic_norm_reps_audio, dim=0)
print(topic_norm_reps_audio.shape)

topic_norm_reps_text = torch.cat(topic_norm_reps_text, dim=0)
print(topic_norm_reps_text.shape)

# probs = audio.softmax(dim=-1).cpu().numpy()
# print(probs)

torch.Size([128, 768])
torch.Size([48640, 768])
torch.Size([48640, 768])


In [80]:
# similarity = text.cpu().numpy().T @  audio.cpu().numpy()


# print(matrix_targets)
# topics_df

In [63]:
# result = similarity.argmax(axis=0)
# ground_truth = torch.arange(similarity.shape[0])

# stats = torch.eq(torch.tensor(result), ground_truth).sum()
# print(stats)

In [81]:
### Validation DF
ids_we_can_use = [m.split('_')[0] for m in matrix_targets]
print("WE CAN USE {} ids".format(len(set(ids_we_can_use))))
val_df = topics_df_targets[topics_df_targets['episode_uri'].isin(ids_we_can_use)]

### TODO: dit naar boven
positive_episodes = val_df.loc[val_df['bin_score'] == 1].copy()
positive_eplist = positive_episodes['episode_uri'].tolist()


for i, row in val_df.iterrows():
    ifor_val = 0
    if row['episode_uri'] in positive_eplist:
        #print("IN TESTLIST")
        ifor_val = 1
    val_df.loc[i,'ep_score'] = ifor_val
    
val_df.ep_score = val_df.ep_score.astype(int)
print(len(val_df))
val_df.head(50)

WE CAN USE 1118 ids


/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


3090


/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,num,unk,episode_uri_time,score,bin_score,episode_uri,time,ep_score
0,9,0,0BrvXC9DNmicnpCtHmFltv_2220,0,0,0BrvXC9DNmicnpCtHmFltv,2220,0
1,9,0,0BrvXC9DNmicnpCtHmFltv_2280,0,0,0BrvXC9DNmicnpCtHmFltv,2280,0
2,9,0,0BrvXC9DNmicnpCtHmFltv_3120,0,0,0BrvXC9DNmicnpCtHmFltv,3120,0
3,9,0,0BrvXC9DNmicnpCtHmFltv_3180,0,0,0BrvXC9DNmicnpCtHmFltv,3180,0
15,9,0,0YYgNb2r5zngCM5VLMpqEb_60,0,0,0YYgNb2r5zngCM5VLMpqEb,60,0
22,9,0,14RFTGKJ9RQlxtJJ6MhNXs_0,0,0,14RFTGKJ9RQlxtJJ6MhNXs,0,1
23,9,0,14RFTGKJ9RQlxtJJ6MhNXs_240,0,0,14RFTGKJ9RQlxtJJ6MhNXs,240,1
24,9,0,14RFTGKJ9RQlxtJJ6MhNXs_300,1,1,14RFTGKJ9RQlxtJJ6MhNXs,300,1
25,9,0,14RFTGKJ9RQlxtJJ6MhNXs_420,0,0,14RFTGKJ9RQlxtJJ6MhNXs,420,1
29,9,0,1jvy4Usn4Z8awRi3gpw0iQ_840,0,0,1jvy4Usn4Z8awRi3gpw0iQ,840,1


In [71]:
# # Create text embeddings of queries
query_field = 'query'
# query_nr = 0
# query = topics_df[query_field].iloc[query_nr]


queries = topics_df[query_field].tolist()
tokenized_queries = tokenizer(
    queries, padding=True, truncation=True, max_length=32, return_tensors='pt', return_token_type_ids=True,
)


query_yamnets = []
query_lengths = []
for idx, row in topics_df.iterrows():
    query_num = row.num

    query_embed_path  = os.path.join(conf.yamnet_query_embed_path, str(query_num) + ".h5")
    inbetween_yamnet_embeds = pd.read_hdf(query_embed_path)

    query_lengths.append(len(inbetween_yamnet_embeds))
    tmp = torch.Tensor(inbetween_yamnet_embeds.values)

    query_yamnets.append(tmp)
    
    
    
    #lengths.append(len(example[1]))

padded_query_yamnets = pad_sequence(query_yamnets, batch_first=True)




In [72]:


query_text = []

query_audio = []
query_field = 'query'

with torch.no_grad():
    # print("[del] get embeds: ")
    reps_sentences = full_model.text_model(tokenized_queries)['sentence_embedding']
    reps_audio = full_model.audio_model((padded_query_yamnets, query_lengths))


    audio_logits =  (reps_audio @ reps_sentences.t()) * CFG.scale
    text_logits = audio_logits.t()

    audio_logits = audio_logits / audio_logits.norm(dim=1, keepdim=True)
    text_logits = text_logits / text_logits.norm(dim=1, keepdim=True)

    #probs = audio_logits.softmax(dim=-1).cpu().numpy()

    query_text_logits = text_logits
    query_audio_logits = audio_logits
    
    query_norm_reps_audio = reps_audio / reps_audio.norm(dim=1, keepdim=True)
    query_norm_reps_text = reps_sentences / reps_sentences.norm(dim=1, keepdim=True)
#     print(probs)



In [73]:
print(query_text_logits)
print(query_text_logits.shape)


print(query_audio_logits)
print(query_audio_logits.shape)

tensor([[ 0.5415, -0.0309,  0.0479,  ..., -0.0179,  0.0459, -0.0405],
        [ 0.0966,  0.0405,  0.0069,  ...,  0.0094, -0.0055, -0.0380],
        [-0.0623, -0.0123,  0.1690,  ...,  0.1604,  0.3965,  0.0594],
        ...,
        [-0.0811, -0.1044,  0.0104,  ...,  0.3829,  0.0840,  0.0459],
        [-0.0662, -0.0360,  0.0052,  ...,  0.2198,  0.2680,  0.0204],
        [-0.0648, -0.0222,  0.0556,  ...,  0.0734,  0.5371,  0.2301]])
torch.Size([50, 50])
tensor([[ 0.6770,  0.1867, -0.0681,  ..., -0.0441, -0.0500, -0.0564],
        [-0.1501,  0.3039, -0.0521,  ..., -0.2201, -0.1053, -0.0751],
        [ 0.1008,  0.0223,  0.3105,  ...,  0.0095,  0.0066,  0.0815],
        ...,
        [-0.0226,  0.0183,  0.1771,  ...,  0.2105,  0.1677,  0.0646],
        [ 0.0286, -0.0053,  0.2153,  ...,  0.0227,  0.1005,  0.2326],
        [-0.0306, -0.0445,  0.0393,  ...,  0.0151,  0.0093,  0.1213]])
torch.Size([50, 50])


In [74]:
print(query_norm_reps_audio.shape)
print(query_norm_reps_text.shape)

torch.Size([50, 768])
torch.Size([50, 768])


In [75]:
from torch import topk
k = 50

pred_episodes = {}

similarity = 100 * query_norm_reps_text @ topic_norm_reps_text.T
probs = similarity.softmax(dim=-1).cpu()

top_probs, top_labels = probs.topk(k, dim=-1)

for num in range(len(topics_df)):
    query = topics_df[query_field][num]
    pred_ind = top_labels[num].tolist()
    
    #pred_targs = matrix_targets[pred_ind]
    pred_epis = [matrix_targets[val].split('_')[0] for val in pred_ind]
    
    tmp = val_df[(val_df.num == num) & (val_df.ep_score==1)]
    tmp = tmp['episode_uri'].tolist()
    num_episodes_relevant = len(set(tmp))
    
    ep_scores = []
    for episode_uri in pred_epis:
        ep_score_row = val_df.loc[val_df['episode_uri'] == episode_uri]
        if len(ep_score_row) > 0 and ep_score_row.num.values[0] == num:
            ep_scores.append(1)
        else:
            ep_scores.append(0)
    
    pred_episodes[query] = {}
    pred_episodes[query]['episodes'] = [matrix_targets[val].split('_')[0] for val in pred_ind]
    pred_episodes[query]['ep_score'] = ep_scores
    
    pred_episodes[query]['prec@3'] = precision_at_k(ep_scores, 3)
    pred_episodes[query]['prec@10'] = precision_at_k(ep_scores, 10)
    pred_episodes[query]['prec@30'] = precision_at_k(ep_scores, 30)
    

    targets = [[1] * num_episodes_relevant + [0] * (len(ep_scores) - num_episodes_relevant)]
    ndcg_ep_score = ndcg_score(targets, [ep_scores], k=30)
    pred_episodes[query]['ndc'] = ndcg_ep_score
    print("done query {}, p@10 {}, ndcg: {}".format(num, pred_episodes[query]['prec@10'], ndcg_ep_score))




# text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
# top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)
# bels = text_probs.cpu().topk(5, dim=-1)



done query 0, p@10 0.0, ndcg: 0.0
done query 1, p@10 0.0, ndcg: 0.0
done query 2, p@10 0.0, ndcg: 0.0
done query 3, p@10 0.0, ndcg: 0.0
done query 4, p@10 0.0, ndcg: 0.0
done query 5, p@10 0.0, ndcg: 0.0
done query 6, p@10 0.0, ndcg: 0.0
done query 7, p@10 0.0, ndcg: 0.0
done query 8, p@10 0.0, ndcg: 0.0
done query 9, p@10 0.0, ndcg: 0.3204893525091456
done query 10, p@10 0.0, ndcg: 0.35256225206696357
done query 11, p@10 0.0, ndcg: 0.44478760329260114
done query 12, p@10 0.0, ndcg: 0.0
done query 13, p@10 0.0, ndcg: 0.210595856149106
done query 14, p@10 0.0, ndcg: 0.35256225206696357
done query 15, p@10 0.0, ndcg: 0.539114729978889
done query 16, p@10 0.0, ndcg: 0.5110239482525717
done query 17, p@10 0.1, ndcg: 0.3193426238738313
done query 18, p@10 0.0, ndcg: 0.3924713158125279
done query 19, p@10 0.0, ndcg: 0.23449324614162684
done query 20, p@10 0.0, ndcg: 0.36659118107830313
done query 21, p@10 0.0, ndcg: 0.2824575001538783
done query 22, p@10 0.0, ndcg: 0.2660517954972375
done qu

In [141]:
# pred_episodes

{'trump call ukrainian president': ['75cpxDXpVz9ScTVNIWz5d6',
  '20Rxox5JAR894PcIMGpsV9',
  '75cpxDXpVz9ScTVNIWz5d6',
  '3QGmWCiVOsKzY85lnSglF6',
  '20Rxox5JAR894PcIMGpsV9',
  '20Rxox5JAR894PcIMGpsV9',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '20Rxox5JAR894PcIMGpsV9',
  '75cpxDXpVz9ScTVNIWz5d6',
  '20Rxox5JAR894PcIMGpsV9',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '20Rxox5JAR894PcIMGpsV9',
  '20Rxox5JAR894PcIMGpsV9',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '3QGmWCiVOsKzY85lnSglF6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '20Rxox5JAR894PcIMGpsV9',
  '3QGmWCiVOsK

In [78]:
# val_df.head(3)
topics_df

,num,query,type,description
0,9,trump call ukrainian president,topical,The White House released a rough transcript of...
1,10,boeing 737 crash causes,topical,The Boeing 737 MAX passenger airliner was grou...
2,11,how to cook turkey,topical,I’m looking for podcasts that talk about how t...
3,12,imran khan career,topical,Imran Khan is the 22nd Prime Minister of Pakis...
4,13,drug addiction recovery,topical,I’m interested in hearing personal stories of ...
5,14,near death experiences,topical,I wonder if people have shared near-death expe...
6,15,podcast about podcasts,topical,I would like to start a podcast. What equipme...
7,16,causes and prevention of wildfires,topical,"2019 saw a large number of wildfires, in Austr..."
8,17,time between meetings,topical,I am looking for some ideas for things to do d...
9,18,women in stem,topical,I’d like to hear personal stories from women i...


In [58]:
from sklearn.metrics import ndcg_score

def precision_at_k(predicted, k):
    if k == 0:
        print("precision from k ===0?")
        raise
    pred = (predicted[:k])
    relevant = np.sum(pred)
    num_recommended = float(k)
    p_at_k = relevant / num_recommended
    return p_at_k

# Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)
def recall_at_k(predicted, num_relevant, k):
    if num_relevant == 0:
        return None
    pred = (predicted[:k])
    relevant = np.sum(pred)
    r_at_k = relevant / num_relevant

    return r_at_k


scores = [[1,1,1,0,1]]
targets = [[1,1,1,1,0]]

print(precision_at_k(scores[0], 5))
print(recall_at_k(scores[0], 0, 2))
ndcg = ndcg_score(targets, scores, k=3)
print(ndcg)
# print(es_url)

0.8
None
0.75


In [157]:

# get score from val df

3QGmWCiVOsKzY85lnSglF6


/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,num,unk,episode_uri_time,score,bin_score,episode_uri,time,ep_score
4767,34,0,75cpxDXpVz9ScTVNIWz5d6_420,0,0,75cpxDXpVz9ScTVNIWz5d6,420,0
5464,35,0,20Rxox5JAR894PcIMGpsV9_1020,3,1,20Rxox5JAR894PcIMGpsV9,1020,1
5465,35,0,20Rxox5JAR894PcIMGpsV9_1380,3,1,20Rxox5JAR894PcIMGpsV9,1380,1
7540,13,0,3QGmWCiVOsKzY85lnSglF6_1980,0,0,3QGmWCiVOsKzY85lnSglF6,1980,0
7541,13,0,3QGmWCiVOsKzY85lnSglF6_2040,0,0,3QGmWCiVOsKzY85lnSglF6,2040,0
7542,13,0,3QGmWCiVOsKzY85lnSglF6_2100,0,0,3QGmWCiVOsKzY85lnSglF6,2100,0
7543,13,0,3QGmWCiVOsKzY85lnSglF6_2160,0,0,3QGmWCiVOsKzY85lnSglF6,2160,0


In [85]:
test = '-'.join(targets)
print(test)

64rTKUGbw1Fkjr3yN7IGL3_3540.0-6FyMGNZKdEIo1AZ5UZs8E6_660.0-0rsRrOjO55SwcGdxpVZpe2_960.0-1XosmQL9gxzVtv0oaAiL3p_2280.0-3zjQsCDgo9uiXDELgLcMYl_3420.0-1GybaPM4vGSmjUX1yFaGSp_1680.0-5YxmR8C0hrjpucr2Ighrko_2640.0-6WK4KpXAsg0D4HJIE8taeq_660.0-6Sz92bNhhnhpQv6lsWL00m_1200.0-10eCpLmJlZ2GveeyX9i3L6_4260.0-5mNyYNul55HkYjwLGfSynh_780.0-3NDGdS3azh9BdQZupmYg8u_960.0-43Vyj3DJAPhvz5XBbgVrAf_2640.0-0wx64UZeGyQ9dTXCk55QHJ_1920.0-1dCvMBsI1lDV5hBatXaFe6_1380.0-6KbD6wm9oZZBGEV94DS9u2_2040.0-1blRzr2pGh0nUAFTVpX7A8_2400.0-4FcseUnGpiuYQQ8CbnM5hk_4500.0-6pigj9rPcgHezUHxQBjFUl_2040.0-2t3Y7Ka1rPJuCCH7vNtiPP_3420.0-2JctcMvBIa6IxxUsnaCfGn_5040.0-5H5WP4sJO5LSFItqbHQyRL_0.0-7KoqVqmBAm44b8E1wmpTRi_120.0-3bgdHb82lmmudMYCm9T4kT_240.0-1dCvMBsI1lDV5hBatXaFe6_3000.0-57tCBGznL6NvNTVwjtw5O7_1320.0-7ok6wB1WviyXvxba0wdb5q_3840.0-2UJznCI45meWpPK5k7YnMZ_1620.0-5rtm2fbnexgkm9ghNHLT06_660.0-4PKizvgQqYKuqiwt7uFN1V_2460.0-5YDBJbpyYqmCjcPYjwcxPh_1980.0-3bi8XhRV6nkWpTiPJr4gcU_2280.0-1rNohl5UGFeokrmAXE2NGD_2400.0-4Fb48WgJPPg86yguJTDXg